# Running Jupyter Notebook using python virtual environment

*David Lawrence Nov. 14, 2019*

This is how you can setup your JLab jupyterhub notebook to run inside a python virtual environment. This is useful for installing python packages via pip from your own account without needing admin privliges (which you don't have). The following will need to be run from a terminal on a CUE computer. I recommend doing this via a terminal launched via jupyterhub since you can use a python version available there that should have all necessary system libraries installed.

1. The JLab firewall will prevent pip from downloading packages using default options. The easiest thing to do is setup a pip configuration file in the home directory of your CUE account that sets the necessary options so you don't have to type them with every pip command. Create a file named "~/.config/pip/pip.conf" and write the following into it:

<pre>
<p style="background-color:lightgrey;color:black">;
; Options to allow pip install to work from behind
; the hallgw firewall
;
   
[install]
trusted-host =
      pypi.org
      files.pythonhosted.org
  
[search]
trusted-host =
      pypi.org
      files.pythonhosted.org
</p>
</pre>

2. NOTE: To create a venv that works with jupyterhub you **must** use the /opt/conda/bin/python3 version and **NOT** the one installed in /apps. This directory is not available on the ifarms so you must execute the commands below in a terminal launched in jupyterhub itself. However, if you wish to run the venv on the ifarm machines (including sciml machines), then you should use the /apps version and run these commands from an ifarm machine.

Create a python virtual environment in the directory of your choosing with:

   `/opt/conda/bin/python3 -m venv venv`
   
   or, if running on an ifarm:
   
   `module load python/3.7.3
 python3.7 -m venv venv`


3. Source the new environment so it will be used for all subsequent pip commands:

   `source venv/bin/activate.csh`

  or

   `source venv/bin/activate`


4. Upgrade the pip package itself to see that everything is working:

   `pip install --upgrade pip`


5. Install python packages you want:

   `pip install tensorflow numpy pandas matplotlib imutils pillow scikit-learn opencv-python pypng`

6. Install the ipykernel into the venv and the venv into your list of kernels:

   `pip install ipykernel`

   `python3 -m ipykernel install --user --name=venv`

7. (OPTIONAL) Configure to use the CUDA libraries so Tensorflow can use GPUs:

    Modern versions of Tensorflow have GPU support built in. To use it though it must know where the CUDA libraries are located. Specify this in the file:

    `~/.local/share/jupyter/kernels/venv/kernel.json`
    
    The content should look something like this with the last line starting with "env" being the thing that needs to be added:
    <pre><p style="background-color:lightgrey;color:black">{
 "argv": [
  "/u/scratch/davidl/venv/bin/python3",
  "-m",
  "ipykernel_launcher",
  "-f",
  "{connection_file}"
 ],
 "display_name": "venv",
 "language": "python",
 "env":{"LD_LIBRARY_PATH":"/apps/cuda/11.2/lib64"}
}</p></pre>

    To test that it found the GPU(s) got on to step 8 and run the following with the new kernel:
    
    `import tensorflow as tf`

    `print('TensorFlow: ' + str(tf.__version__))`

    `physical_devices = tf.config.list_physical_devices()`

    `print(physical_devices)`


8. At this point a new option for "venv" should be there in your launcher for both "Notebook" and "Console". You can verify that it is using your virtual environment by doing something like:

   `import sys
print(sys.path)`


Notes:

* The virtual environment is now part of a "kernel" and you can switch existing notebooks to using it via a menu in the upper right-hand corner of the notebook. (you may need to close and reopen the notebook to see the new kernel in the menu)
* You can list available kernels via command line using "**jupyter kernelspec list**"
* You can remove an old kernels using "**jupyter kernelspec uninstall old-kernel**"
